`November 18, 2022`

In [ ]:
# Import libraries
import numpy as np
import pandas as pd

## **Pandas Merge, Join, Concatenation**

In [ ]:
# Create dummy data

sales = {'Aaron': 110,
         'Bush': 204,
         'Charlie': 436,
         'Diana': 397,
         'Erika': 521
         }

region = {'Aaron': 'Jawa Barat',
          'Bush': 'Sumatera Selatan',
          'Charlie': 'Jawa Barat',
          'Frank': 'Sulawesi Utara',
          'Diana': 'Sumatera Selatan',
          'Erika': 'Kalimatan Barat',
          'Harry': 'Jawa Timur',
          'Katya': 'DKI Jakarta',
          'James': 'Jawa Barat'
        }

In [ ]:
# Membuat DataFrame
df_sales = pd.DataFrame.from_dict(sales, orient='index', columns=['sales'])

df_region = pd.DataFrame.from_dict(region, orient='index', columns=['region'])

display(df_sales)
display(df_region)

,sales
Aaron,110
Bush,204
Charlie,436
Diana,397
Erika,521


,region
Aaron,Jawa Barat
Bush,Sumatera Selatan
Charlie,Jawa Barat
Frank,Sulawesi Utara
Diana,Sumatera Selatan
Erika,Kalimatan Barat
Harry,Jawa Timur
Katya,DKI Jakarta
James,Jawa Barat


In [ ]:
# Cek index df_sales
df_sales.index

Index(['Aaron', 'Bush', 'Charlie', 'Diana', 'Erika'], dtype='object')

In [ ]:
# Cek sales dari Bush
df_sales.loc['Bush']

sales    204
Name: Bush, dtype: int64

**Merge**

- Merge biasa digunakan ketika kita tidak mau menggabungkan DataFrame berdasarkan index.
- Jika nama kolom pada kedua DataFrame berbeda tapi merepresentasikan isi yang sama, maka kita bisa mendefinisikan parameter left_on dan right_on.

In [ ]:
# Menggabungkan df_region dengan df_sales menggunakan .merge()
df_merge = df_region.merge(
                            df_sales,
                            how='left',
                            left_index=True,
                            right_index=True,
                            indicator=True
                            )

df_merge

,region,sales,_merge
Aaron,Jawa Barat,110.0,both
Bush,Sumatera Selatan,204.0,both
Charlie,Jawa Barat,436.0,both
Frank,Sulawesi Utara,NaN,left_only
Diana,Sumatera Selatan,397.0,both
Erika,Kalimatan Barat,521.0,both
Harry,Jawa Timur,NaN,left_only
Katya,DKI Jakarta,NaN,left_only
James,Jawa Barat,NaN,left_only


In [ ]:
# Total sales per region
grouped_region = df_merge.groupby('region').sum().sort_values(by='sales', ascending=False)
grouped_region.reset_index(inplace=True)
grouped_region

,region,sales
0,Sumatera Selatan,601.0
1,Jawa Barat,546.0
2,Kalimatan Barat,521.0
3,DKI Jakarta,0.0
4,Jawa Timur,0.0
5,Sulawesi Utara,0.0


In [ ]:
# Drop kolom _merge
df_merge.drop(columns='_merge', axis=1, inplace=True)

In [ ]:
# Ingin mengecek kontribusi tiap employee terhadap sales di regionnya
emp_contribution = df_merge.merge(
                                    grouped_region,
                                    how='left',
                                    left_on='region',
                                    right_on='region',
                                    suffixes=('_employee', '_region')
                                )

emp_contribution

,region,sales_employee,sales_region
0,Jawa Barat,110.0,546.0
1,Sumatera Selatan,204.0,601.0
2,Jawa Barat,436.0,546.0
3,Sulawesi Utara,NaN,0.0
4,Sumatera Selatan,397.0,601.0
5,Kalimatan Barat,521.0,521.0
6,Jawa Timur,NaN,0.0
7,DKI Jakarta,NaN,0.0
8,Jawa Barat,NaN,546.0


In [ ]:
# Mengubah index menjadi nama tiap employee
emp_contribution = emp_contribution.set_index(df_merge.index)
emp_contribution

,region,sales_employee,sales_region
Aaron,Jawa Barat,110.0,546.0
Bush,Sumatera Selatan,204.0,601.0
Charlie,Jawa Barat,436.0,546.0
Frank,Sulawesi Utara,NaN,0.0
Diana,Sumatera Selatan,397.0,601.0
Erika,Kalimatan Barat,521.0,521.0
Harry,Jawa Timur,NaN,0.0
Katya,DKI Jakarta,NaN,0.0
James,Jawa Barat,NaN,546.0


In [ ]:
# Menambahkan kolom persentase dari sales tiap employee.
    # Untuk mengetahui kontribusi employee terhadap sales dalam persentase

emp_contribution['sales_percentage'] = round(emp_contribution['sales_employee'] / emp_contribution['sales_region'] * 100, 2)
emp_contribution

,region,sales_employee,sales_region,sales_percentage
Aaron,Jawa Barat,110.0,546.0,20.15
Bush,Sumatera Selatan,204.0,601.0,33.94
Charlie,Jawa Barat,436.0,546.0,79.85
Frank,Sulawesi Utara,NaN,0.0,NaN
Diana,Sumatera Selatan,397.0,601.0,66.06
Erika,Kalimatan Barat,521.0,521.0,100.00
Harry,Jawa Timur,NaN,0.0,NaN
Katya,DKI Jakarta,NaN,0.0,NaN
James,Jawa Barat,NaN,546.0,NaN


In [ ]:
# Mengganti missing value dengan 0
emp_contribution = emp_contribution.fillna({
                                            'sales_employee': 0,
                                            'sales_region': 0,
                                            'sales_percentage': 0
})

emp_contribution

,region,sales_employee,sales_region,sales_percentage
Aaron,Jawa Barat,110.0,546.0,20.15
Bush,Sumatera Selatan,204.0,601.0,33.94
Charlie,Jawa Barat,436.0,546.0,79.85
Frank,Sulawesi Utara,0.0,0.0,0.00
Diana,Sumatera Selatan,397.0,601.0,66.06
Erika,Kalimatan Barat,521.0,521.0,100.00
Harry,Jawa Timur,0.0,0.0,0.00
Katya,DKI Jakarta,0.0,0.0,0.00
James,Jawa Barat,0.0,546.0,0.00


**Join**

In [ ]:
# Default how parameter pada .join adalah left
joined_df = df_region.join(df_sales)
joined_df

,region,sales
Aaron,Jawa Barat,110.0
Bush,Sumatera Selatan,204.0
Charlie,Jawa Barat,436.0
Frank,Sulawesi Utara,NaN
Diana,Sumatera Selatan,397.0
Erika,Kalimatan Barat,521.0
Harry,Jawa Timur,NaN
Katya,DKI Jakarta,NaN
James,Jawa Barat,NaN


**Concatenation**

In [ ]:
# Create dummy data
df1 = pd.DataFrame({
                    'Jakarta': [1, 2, 3, 4],
                    'Palangkaraya': [5, 6, 7, 8],
                    'Gorontalo': [9, 10, 11, 12],
                    }, index=[10, 12, 35, 70]
                    )

df2 = pd.DataFrame({
                    'Jakarta': [11, 22, 33, 44],
                    'Palangkaraya': [54, 62, 71, 85],
                    'Gorontalo': [91, 120, 151, 126],
                    }, index=[4, 6, 19, 20]
                    )

df3 = pd.DataFrame({
                    'Jakarta': [21, 43, 23, 64],
                    'Palangkaraya': [14, 62, 91, 89],
                    'Gorontalo': [91, 125, 151, 128],
                    }, index=[24, 56, 16, 40]
                    )

display(df1)
display(df2)
display(df3)

,Jakarta,Palangkaraya,Gorontalo
10,1,5,9
12,2,6,10
35,3,7,11
70,4,8,12


,Jakarta,Palangkaraya,Gorontalo
4,11,54,91
6,22,62,120
19,33,71,151
20,44,85,126


,Jakarta,Palangkaraya,Gorontalo
24,21,14,91
56,43,62,125
16,23,91,151
40,64,89,128


In [ ]:
# Menggabungkan 3 DataFrame berdasarkan axis=0 (penambahan baris)
# Default axis=0
pd.concat([df1, df2, df3]).reset_index(drop=True)

,Jakarta,Palangkaraya,Gorontalo
0,1,5,9
1,2,6,10
2,3,7,11
3,4,8,12
4,11,54,91
5,22,62,120
6,33,71,151
7,44,85,126
8,21,14,91
9,43,62,125


In [ ]:
# Concat berdasarkan axis=1 (penambahan kolom)
pd.concat([df1, df2, df3], axis=1).reset_index(drop=True)

,Jakarta,Palangkaraya,Gorontalo,Jakarta,Palangkaraya,Gorontalo,Jakarta,Palangkaraya,Gorontalo
0,NaN,NaN,NaN,11.0,54.0,91.0,NaN,NaN,NaN
1,NaN,NaN,NaN,22.0,62.0,120.0,NaN,NaN,NaN
2,1.0,5.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,6.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,23.0,91.0,151.0
5,NaN,NaN,NaN,33.0,71.0,151.0,NaN,NaN,NaN
6,NaN,NaN,NaN,44.0,85.0,126.0,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,21.0,14.0,91.0
8,3.0,7.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,64.0,89.0,128.0


### **Pandas Operations**

In [ ]:
joined_df.dropna(inplace=True)
joined_df

,region,sales
Aaron,Jawa Barat,110.0
Bush,Sumatera Selatan,204.0
Charlie,Jawa Barat,436.0
Diana,Sumatera Selatan,397.0
Erika,Kalimatan Barat,521.0


**Columns Operation Using Regular Function**

In [ ]:
# Buat regular function kali 2
def kali_dua(x):
    return x*2

In [ ]:
# Menggunakan regular function yang sudah dibuat
joined_df['sales'].apply(kali_dua)

Aaron       220.0
Bush        408.0
Charlie     872.0
Diana       794.0
Erika      1042.0
Name: sales, dtype: float64

In [ ]:
# Menghitung jumlah baris pada kolom sales
len(joined_df['sales'])

5

In [ ]:
# Menghitung jumlah karakter dari tiap value di kolom region
joined_df['region'].apply(len)

Aaron      10
Bush       16
Charlie    10
Diana      16
Erika      15
Name: region, dtype: int64

**Column Operation Using Lambda**

In [ ]:
# Mengalikan 3 semua nilai pada kolom sales
joined_df['sales'].apply(lambda sales: sales * 3)

Aaron       330.0
Bush        612.0
Charlie    1308.0
Diana      1191.0
Erika      1563.0
Name: sales, dtype: float64

In [ ]:
# Mengembalikan karakter index ketiga pada tiap row di kolom region
joined_df['region'].apply(lambda x: x[3])

Aaron      a
Bush       a
Charlie    a
Diana      a
Erika      i
Name: region, dtype: object

**Math Operations**

In [ ]:
# Berlaku untuk semua operasi dasar math
joined_df['sales_kuadrat'] = joined_df['sales'] * joined_df['sales']
joined_df

,region,sales,sales_kuadrat
Aaron,Jawa Barat,110.0,12100.0
Bush,Sumatera Selatan,204.0,41616.0
Charlie,Jawa Barat,436.0,190096.0
Diana,Sumatera Selatan,397.0,157609.0
Erika,Kalimatan Barat,521.0,271441.0


#### **Pivot Table**

In [ ]:
# Tambahkan kolom cabang
joined_df['branch'] = ['A', 'A', 'B', 'B', 'A']
joined_df

,region,sales,sales_kuadrat,branch
Aaron,Jawa Barat,110.0,12100.0,A
Bush,Sumatera Selatan,204.0,41616.0,A
Charlie,Jawa Barat,436.0,190096.0,B
Diana,Sumatera Selatan,397.0,157609.0,B
Erika,Kalimatan Barat,521.0,271441.0,A


In [ ]:
# Tambahkan kolom bulan
joined_df['month'] = 'July July July July July'.split()
joined_df

,region,sales,sales_kuadrat,branch,month
Aaron,Jawa Barat,110.0,12100.0,A,July
Bush,Sumatera Selatan,204.0,41616.0,A,July
Charlie,Jawa Barat,436.0,190096.0,B,July
Diana,Sumatera Selatan,397.0,157609.0,B,July
Erika,Kalimatan Barat,521.0,271441.0,A,July


In [ ]:
# Pivoting
# 3 parameter utama yang harus dimasukkan saat pivoting di Pandas --> values, index, columns
joined_df.pivot_table(
                        values='sales',
                        index=['region', 'branch'],
                        columns='month'
)

month                     July
region           branch       
Jawa Barat       A       110.0
                 B       436.0
Kalimatan Barat  A       521.0
Sumatera Selatan A       204.0
                 B       397.0

### **Pandas: Data Input & Output**

#### **csv**

**Output**

In [ ]:
joined_df.to_csv('joined_df.csv', index=False)

**Input**

In [ ]:
df_join = pd.read_csv('joined_df.csv')
df_join

,region,sales,sales_kuadrat,branch,month
0,Jawa Barat,110.0,12100.0,A,July
1,Sumatera Selatan,204.0,41616.0,A,July
2,Jawa Barat,436.0,190096.0,B,July
3,Sumatera Selatan,397.0,157609.0,B,July
4,Kalimatan Barat,521.0,271441.0,A,July


#### **excel**

**Output**

In [ ]:
joined_df.to_excel('joined_df.xlsx', 
                    sheet_name='sales_joined',
                    index=False
                    )

**Input**

In [ ]:
df_excel = pd.read_excel('joined_df.xlsx',
                         sheet_name='sales_joined')

df_excel

,region,sales,sales_kuadrat,branch,month
0,Jawa Barat,110,12100,A,July
1,Sumatera Selatan,204,41616,A,July
2,Jawa Barat,436,190096,B,July
3,Sumatera Selatan,397,157609,B,July
4,Kalimatan Barat,521,271441,A,July


#### **JSON**

**Output**

In [ ]:
joined_df.to_json('sales_joined.json',
                  orient='records')

**Input**

In [ ]:
df_json = pd.read_json('sales_joined.json',
                        orient='records')

df_json

,region,sales,sales_kuadrat,branch,month
0,Jawa Barat,110,12100,A,July
1,Sumatera Selatan,204,41616,A,July
2,Jawa Barat,436,190096,B,July
3,Sumatera Selatan,397,157609,B,July
4,Kalimatan Barat,521,271441,A,July


In [ ]:
joined_df.to_json('sales_joined_table.json',
                  orient='table')

In [ ]:
df_json_table = pd.read_json('sales_joined.json',
                              orient='records')

df_json_table

,region,sales,sales_kuadrat,branch,month
0,Jawa Barat,110,12100,A,July
1,Sumatera Selatan,204,41616,A,July
2,Jawa Barat,436,190096,B,July
3,Sumatera Selatan,397,157609,B,July
4,Kalimatan Barat,521,271441,A,July


#### **HTML**

In [ ]:
# Generate table from a website
df_html = pd.read_html('https://www.statista.com/statistics/1104709/coronavirus-deaths-worldwide-per-million-inhabitants/')
df_html

[             Characteristic  Confirmed cases (absolute)  Cases in last 7 days  \
 0                      Peru                     3691213                 47014   
 1                  Bulgaria                     1180636                  5554   
 2    Bosnia and Herzegovina                      379982                   850   
 3                   Hungary                     1932788                  4663   
 4                   Georgia                     1662299                     0   
 ..                      ...                         ...                   ...   
 149             South Sudan                       17733                     0   
 150                    Chad                        7427                     1   
 151                   China                     2170811                 23481   
 152                 Burundi                       43060                   266   
 153             New Zealand                        5814                    24   
 
      Confirme

In [ ]:
df_covid = df_html[0]
df_covid

,Characteristic,Confirmed cases (absolute),Cases in last 7 days,Confirmed deaths (absolute),Population (in millions),Deaths in last 7 days,Deaths per million (last 7 days),Daily increase (# deaths),Deaths per million (total)
0,Peru,3691213,47014,213714,32.97,121,3.67,19,6481.71
1,Bulgaria,1180636,5554,37279,6.93,18,2.60,2,5376.25
2,Bosnia and Herzegovina,379982,850,15810,3.28,2,0.61,0,4818.92
3,Hungary,1932788,4663,46661,9.75,14,1.44,0,4785.86
4,Georgia,1662299,0,16844,3.71,0,0.00,0,4535.27
...,...,...,...,...,...,...,...,...,...
149,South Sudan,17733,0,138,11.19,0,0.00,0,12.33
150,Chad,7427,1,193,16.43,0,0.00,0,11.75
151,China,2170811,23481,14649,1410.93,16,0.01,2,10.38
152,Burundi,43060,266,38,11.89,0,0.00,0,3.20


In [ ]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Characteristic                    154 non-null    object 
 1   Confirmed cases (absolute)        154 non-null    int64  
 2   Cases in last 7 days              154 non-null    int64  
 3   Confirmed deaths (absolute)       154 non-null    int64  
 4   Population (in millions)          154 non-null    float64
 5   Deaths in last 7 days             154 non-null    int64  
 6   Deaths per million (last 7 days)  154 non-null    float64
 7   Daily increase (# deaths)         154 non-null    int64  
 8   Deaths per million (total)        154 non-null    float64
dtypes: float64(3), int64(5), object(1)
memory usage: 11.0+ KB


In [ ]:
# Alternatif lain
import warnings
warnings.filterwarnings('ignore')

pd.DataFrame(df_html, dtype=object)[0][0]

,Characteristic,Confirmed cases (absolute),Cases in last 7 days,Confirmed deaths (absolute),Population (in millions),Deaths in last 7 days,Deaths per million (last 7 days),Daily increase (# deaths),Deaths per million (total)
0,Peru,3691213,47014,213714,32.97,121,3.67,19,6481.71
1,Bulgaria,1180636,5554,37279,6.93,18,2.60,2,5376.25
2,Bosnia and Herzegovina,379982,850,15810,3.28,2,0.61,0,4818.92
3,Hungary,1932788,4663,46661,9.75,14,1.44,0,4785.86
4,Georgia,1662299,0,16844,3.71,0,0.00,0,4535.27
...,...,...,...,...,...,...,...,...,...
149,South Sudan,17733,0,138,11.19,0,0.00,0,12.33
150,Chad,7427,1,193,16.43,0,0.00,0,11.75
151,China,2170811,23481,14649,1410.93,16,0.01,2,10.38
152,Burundi,43060,266,38,11.89,0,0.00,0,3.20


In [ ]:
df_covid[df_covid['Characteristic'] == 'Indonesia']

,Characteristic,Confirmed cases (absolute),Cases in last 7 days,Confirmed deaths (absolute),Population (in millions),Deaths in last 7 days,Deaths per million (last 7 days),Daily increase (# deaths),Deaths per million (total)
81,Indonesia,6116347,18419,156806,273.52,40,0.15,8,573.28
